# UHFQA

Just like the driver for the *HDAWG* in the previous example, we now use the `tk.UHFQA` instrument driver.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import qcodes as qc
import zhinst.qcodes as ziqc

uhfqa = ziqc.UHFQA("qa1", "dev2266", interface="1gbe", host="10.42.0.226")

Successfully connected to data server at 10.42.0.2268004 api version: 6
Successfully connected to device DEV2266 on interface 1GBE
Connected to: Zurich Instruments UHFQA (serial:dev2266, firmware:65939) in 1.02s


In [2]:
print([k for k in uhfqa.submodules.keys()])
print([k for k in uhfqa.parameters.keys()])

['channels', 'awg', 'stats', 'oscs', 'triggers', 'status', 'dios', 'auxins', 'system', 'sigins', 'sigouts', 'features', 'auxouts', 'qas']
['IDN', 'crosstalk_matrix', 'result_source', 'integration_time', 'averaging_mode', 'clockbase']


## AWG Core of the UHFQA

Also the *UHFQA* features one *AWG Core*. 

In [3]:
print([k for k in uhfqa.awg.parameters.keys()])

['outputs', 'output1', 'output2', 'gain1', 'gain2']


## Readout Channels of the UHFQA

The *UHFQA* comes with signal processing streams for up to ten channels in parallel. The settings for the readout are grouped by channel in a list of all ten `channels`. Each item in the `channels` property of the *UHFQA* is an *Instrument Channel* that represent the signal processing path for one of the ten channels.

In [4]:
print([k for k in uhfqa.channels[0].parameters.keys()])

['rotation', 'threshold', 'readout_frequency', 'readout_amplitude', 'phase_shift', 'result', 'enabled']


Each of the channels follows the following signal processing steps:

1. Demodulation of the input signal
2. Rotation in the complex plane
3. Thresholding for binary result values


The values for the rotation and thresholding stages can be set using the `rotation` and `threshold` parameter of the *channel*. 

The standard mode for the demodulation of input signals is the *weighted integration* mode. This corresponds to setting the integration weights for the two quadratures of the input signal to oscillate at a given demodulation frequency. When enabling the weighted integration with `ch.enable()`, the integration weights for the two quadratures are set. The demodulation frequency is set to the parameter `readout_frequency`.

Enabling weighted integration for the first four channels of the *UHFQA* and setting their demodulation frequency could look like this: 

In [5]:
freqs = [85.6e6, 101.3e6, 132.8e6]

for ch in uhfqa.channels[:3]:
    ch.enable()
    ch.readout_frequency(freqs[ch.index])

The resut vector of each channel can be retrieved from the instrument by calling the read-only parameter *result*.

In [6]:
print(uhfqa.channels[0].result.__doc__)

Node: qas/0/result/data/0/wave
Description: Returns the result vector of the readout channel.
Type: Numpy array
Properties: Read
Unit: None


Parameter class:

* `name` result
* `label` Result
* `unit` None
* `vals` None


## Readout Parameters

There are readout parameters taht are not specific to one isngle channel but affect all ten readout channels. These are

* the `integration_time`: the time in seconds used for integrating the input signals
* the `result_source` lets the user select at which point in the signal processing path the `result` value should be taken
* the `averaging_mode` specifies if the hardware averages on the device should be taken in a *sequential* or *cyclic* way 
* the `crosstalk_matrix` specifies a 10 x 10 matrix that can be calibrated to compensate for crosstalk betweeen the channels

These three *parameters* are attributes of the *UHFQA* instrument driver.

In [7]:
print(uhfqa.integration_time.__doc__)

The integration time used for demodulation in seconds. Can be up to 2.27 us when using weighted integration and up to 50 us in spectroscopy mode.

Parameter class:

* `name` integration_time
* `label` Integration Time
* `unit` 
* `vals` <Numbers 0<=v<=5e-05>


In [8]:
print(uhfqa.result_source.__doc__)

The signal source for QA Results. Has to be one of ['Crosstalk', 'Integration', 'Threshold', 'Crosstalk Correlation', 'Threshold Correlation', 'Rotation'].

Parameter class:

* `name` result_source
* `label` Result Source
* `unit` 
* `vals` <Enum: {'Crosstalk', 'Threshold', 'Threshold Correlation', 'Crosstalk Correlation', 'Integration', 'Rotation'}>


In [9]:
print(uhfqa.averaging_mode.__doc__)

Selects the order of the result logger. One of {'Cyclic', 'Sequential'}.

Parameter class:

* `name` averaging_mode
* `label` Averaging Mode
* `unit` 
* `vals` <Enum: {'Sequential', 'Cyclic'}>


In [10]:
print(uhfqa.crosstalk_matrix.__doc__)

The 10x10 crosstalk suppression matrix that multiplies the 10 signal paths. Can be set only partially.

Parameter class:

* `name` crosstalk_matrix
* `label` Crosstalk Matrix
* `unit` 
* `vals` None


Other important readout parameters can be accessed through the *nodetree*, for example the 

* *result length*: the number of points to acquire
* *result averages*: the number of hardware averages

In [11]:
print(uhfqa.qas[0].result.length.__doc__)

* `Node`: /DEV2266/QAS/0/RESULT/LENGTH
* `Description`: The time duration of each capture in samples. A maximum of 4096 samples can be captured, which corresponds to 2.3 µs.
* `Properties`: Read, Write, Setting
* `Type`: Integer (64 bit)
* `Unit`: None


Parameter class:

* `name` length
* `label` length
* `unit` 
* `vals` None


In [12]:
print(uhfqa.qas[0].result.averages.__doc__)

* `Node`: /DEV2266/QAS/0/RESULT/AVERAGES
* `Description`: log2 of the number of averages to perform, i.e. 0 means no averaging, 1 means 2 values are averaged, etc. Maximum setting is 15 meaning 2^15 values are averaged.
* `Properties`: Read, Write, Setting
* `Type`: Integer (64 bit)
* `Unit`: None


Parameter class:

* `name` averages
* `label` averages
* `unit` 
* `vals` None


## *arm* the UHFQA Readout

The `arm(...)` method of the *UHFQA* prepares the device for data acquisition. It enables the *Results Acquisition* and resets the acquired points to zero. This should be done before every measurement. The method also includes a shortcut to setting the values *result length* and *result averages*. They can be specified as keyword arguments. If the keyword arguemnts are not specified, nothing is changed.

In [13]:
uhfqa.arm(length=1e3, averages=2**5)